In [4]:
import requests
import json
import os 
import pandas as pd
from dataClass import DataTable
from main import list_files_in_folder
from dotenv import load_dotenv
from tqdm import tqdm
load_dotenv()


def LamAPI(cell_content):
    
    url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
    params = {
        'name': f'{cell_content}',
        'token': os.getenv("LAMAPI_KEY"),
        'kg': 'wikidata',
        'fuzzy': 'True'
    }
    headers = {'accept': 'application/json'}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # Process the JSON data here
    else:
        print("Error:", response.status_code)
    
    return data

In [2]:
gt_path = 'data/HardTablesR1/DataSets/HardTablesR1/Valid/gt/cea_gt.csv'
tables_path = 'data/HardTablesR1/DataSets/HardTablesR1/Valid/tables'

ground_truth = pd.read_csv(gt_path, header=None)
tables = list_files_in_folder(tables_path)

In [ ]:
results = {}
for table_path in tqdm(tables):
    t = DataTable(table_path)
    gt = ground_truth[ground_truth[0]==t.name]
    for i, row in gt.iterrows():
        cell_content = t.data.iloc[row[1]-1, row[2]]
        retrieved_list = LamAPI(cell_content)
        id_to_remove = row[3].split('/')[-1]
        ls = [d for d in retrieved_list if d.get('id') != id_to_remove]
        results[t.name] = {f"{(row[1], row[2])}": {'cell': cell_content,
                                              'retrieved_list': ls}}